In [7]:
import re
import csv
import string

In [15]:
import nltk
nltk.download('all')
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from collections import Counter

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\abc.zip.
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers\averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping grammar

Auxiliar functions
- remove_scenes_actions
- extract_trigrams

In [16]:
def remove_scenes_actions(line):
    new = ""
    found = False
    for letter in line:
        if letter == "(" or letter =="[":
            found = True
        if letter == ")" or letter =="]":
            found = False

        #outside brackets
        if found == False and letter!=")" and letter!="]":
            new = new + letter

    
    return(new)

In [17]:
def extract_trigrams(sentence):
    # Step 1
    words_list = sentence.split(" ")

    # Step 2
    trigrams_list = []

    # Step 3
    for i in range(0 , len(words_list) - 2):
        if words_list[i]!='' and words_list[i+1]!='' and words_list[i+2]!='':
            trigram = [words_list[i],words_list[i+1],words_list[i + 2]]
            trigrams_list.append(trigram)
    return trigrams_list

In [18]:
file = open("friends.csv", 'r', encoding="utf8")
csvreader = csv.reader(file)
data = list(csvreader)

characters =["Rachel", "Monica", "Joey", "Chandler", "Ross", "Phoebe"]

stop_words = set(stopwords.words('english'))

*************** UNIGRAMS ***********************************************

In [23]:
for character in characters:
        lines = []
        sp_character = []
        for row in data:
            if row[-2] == character:
                #we only want dialogue
                str = remove_scenes_actions(row[-1])
                lines.append(str)

        #create auxiliar vector WITHOUT punctuation for TF-IDF purposes
        words = []

        for line in lines:
            #remve punctuation
            line = line.translate(str.maketrans('', '', string.punctuation))
            #sp_character.append(line)
            
            line = word_tokenize(line)

            for word in line:
                words.append(word.lower())

        counts = Counter(words)
        counts = sorted(counts.items(), key=lambda x: x[1], reverse=True)
        print(character)

        for elem in counts:
            if elem[0] not in stop_words:
                print(elem)
        print("***************************************")

Rachel
('’', 3469)
('oh', 1707)
('know', 1011)
('okay', 815)
('yeah', 730)
('well', 727)
('na', 595)
('right', 525)
('gon', 509)
('ross', 485)
('god', 422)
('really', 401)
('get', 401)
('like', 389)
('mean', 388)
('im', 365)
('think', 339)
('got', 337)
('go', 329)
('joey', 324)
('hey', 322)
('hi', 320)
('come', 278)
('would', 265)
('look', 261)
('sorry', 259)
('want', 248)
('guys', 246)
('one', 245)
('see', 243)
('dont', 241)
('uh', 233)
('going', 215)
('phoebe', 215)
('monica', 213)
('great', 206)
('good', 205)
('yes', 198)
('tell', 196)
('yknow', 184)
('little', 178)
('honey', 178)
('love', 177)
('back', 171)
('thank', 170)
('wait', 169)
('time', 167)
('ok', 163)
('youre', 156)
('thats', 148)
('wow', 145)
('thing', 145)
('say', 143)
('please', 139)
('said', 138)
('chandler', 137)
('let', 137)
('something', 137)
('umm', 137)
('could', 134)
('take', 121)
('ohh', 121)
('never', 119)
('guy', 118)
('make', 116)
('actually', 114)
('us', 114)
('big', 113)
('maybe', 113)
('even', 111)
('beli